# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 12
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiDi

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    
        num. electrons       : 60
        spin polarization    : none
        temperature          : 0.001 Ha
        smearing             : DFTK.Smearing.FermiD

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355325075276                   -1.10    6.2    175ms
  2   -9.356799102300       -2.83       -1.43    1.0   76.5ms
  3   -9.357073895817       -3.56       -2.77    1.4   83.6ms
  4   -9.357110357668       -4.44       -2.98    9.0    207ms
  5   -9.357110871749       -6.29       -3.12    1.1   77.2ms
  6   -9.357111187306       -6.50       -3.27    1.1   77.5ms
  7   -9.357111383466       -6.71       -3.44    2.1   88.3ms
  8   -9.357111453390       -7.16       -3.59    1.0   76.0ms
  9   -9.357111497850       -7.35       -3.88    1.0   76.2ms
 10   -9.357111506367       -8.07       -4.18    1.1    120ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.75218005932                   -0.97    7.0    390ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     7
│     1
│     2
│     1
│     6
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.79160979434       -1.40       -1.33    2.5    257ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      6
│     13
│      4
│      8
│      3
│     11
│      1
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -18.79247471820       -3.06       -2.42    6.0    352ms
  4   -18.79260215520       -3.89       -2.90    5.4    397ms
  5   -18.79260593512       -5.42       -3.20    2.8    267ms
  6   -18.79260631110       -6.42       -3.50    3.1    256ms
  7   -18.79260637474       -7.20       -4.15    3.9    279ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     15
│      2
│     16
│      2
│     15
│     12
│     14
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55003382085                   -0.84    9.8    2.09s
  2   -37.55805031673       -2.10       -1.22    2.8    878ms
  3   -37.56122220892       -2.50       -2.19    6.8    1.21s
  4   -37.56484792820       -2.44       -2.24   15.0    1.71s
  5   -37.56490330724       -4.26       -2.39    3.9    959ms
  6   -37.56493623435       -4.48       -2.97    3.5    918ms
  7   -37.56494547332       -5.03       -3.19    6.6    1.22s
  8   -37.56494943702       -5.40       -3.66    2.1    1.33s
  9   -37.56494985392       -6.38       -4.29    5.2    1.31s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355222165102                   -1.09    6.0    183ms
  2   -9.356828061730       -2.79       -1.91    1.0   65.8ms
  3   -9.357077841116       -3.60       -2.53    5.6    142ms
  4   -9.357094049363       -4.79       -2.63    2.9    102ms
  5   -9.357111072018       -4.77       -3.53    1.0   65.2ms
  6   -9.357111442720       -6.43       -3.92    4.4    119ms
  7   -9.357111505758       -7.20       -4.54    3.5    107ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54838406027                   -0.84   10.2    1.70s
  2   -37.52134429189   +   -1.57       -1.53    2.9    1.23s
  3   -4.475674411853   +    1.52       -0.31   10.0    2.09s
  4   -37.54201344428        1.52       -1.67    8.1    2.03s
  5   -37.52347914964   +   -1.73       -1.69    3.5    1.10s
  6   -37.54548514125       -1.66       -1.87    1.5    772ms
  7   -37.37252078179   +   -0.76       -1.42    4.6    1.29s
  8   -37.53082250191       -0.80       -1.77    4.9    1.77s
  9   -37.56400089001       -1.48       -2.33    2.6    854ms
 10   -37.56416220859       -3.79       -2.37    3.4    1.12s
 11   -37.56491273333       -3.12       -2.92    1.9    867ms
 12   -37.56483608714   +   -4.12       -2.92    3.2    1.22s
 13   -37.56494001345       -3.98       -3.24    1.8    844ms
 14   -37.56494743394       -5.13       -3.59    2.8    939ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.